In [1]:
import xarray as xr

from dask import delayed, compute
from dask.distributed import Client
import numpy as np

import os
from pathlib import Path
os.chdir('/nesi/nobackup/niwa00004/meyerstj/data/SMAP')

import smap_analysis.utils as sa

In [2]:
os.environ['MALLOC_TRIM_THRESHOLD_'] = '0'

### SMAP data guides
- <b>User guide for SMAP L3 SPM_E (9km)</b>: https://nsidc.org/sites/default/files/documents/user-guide/spl3smp_e-v006-userguide.pdf
- <b>Paper doing validation on SMAP, has good explanations</b>: https://www.mdpi.com/2071-1050/15/11/9112
- <b> Summary data table of all SMAP products</b>: https://nsidc.org/data/user-resources/help-center/what-data-subsetting-reformatting-and-reprojection-services-are-available-smap-data
- <b> `wget` how to </b> https://nsidc.org/data/user-resources/help-center/programmatic-data-access-guide
- <b>JPL SMAP summary</b>: https://smap.jpl.nasa.gov/data/
- <b> `SMAP DATA` </b>: https://n5eil01u.ecs.nsidc.org/SMAP/

### Projections
- <b> EASE2Grid projections done in this notebook</b>: https://github.com/nsidc/smap_python_notebooks/blob/main/notebooks/2.0%20Read%20and%20Plot%20SMAP%20data.ipynb 


In [3]:
# all_downloaded smap files 
# This was done with the "download_smap.py" script.
smap_files = np.sort(list(
    Path('n5eil01u.ecs.nsidc.org/SMAP/SPL3SMP_E.006/').glob('*/*.h5')
))
    

# Try with dask

In [ ]:
sa.extract_smap('2016-11-07 00:00:00', force = True, save = False)

In [4]:
client = Client(n_workers = 10, memory_limit = '30GB', threads_per_worker = 1, processes = True)

In [5]:
%%time
delayed_list = []
for f in smap_files:
    timestamp = sa.extract_timestamp_from_fpath(f)
    _,__, oname = sa.create_smap_fpaths(timestamp)
    
    oname = Path(oname)
    if not oname.exists():
        ds = delayed_list.append(delayed(sa.extract_smap)(timestamp, force= False, save=True, extract_kwargs = {'domain': (slice(-33, -55), slice(165, 185))}))


CPU times: user 1.5 s, sys: 200 ms, total: 1.7 s
Wall time: 2.34 s


In [ ]:
compute(delayed_list) 

/nesi/nobackup/niwa00004/meyerstj/data/SMAP/venv/lib/python3.11/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 53.38 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/nesi/nobackup/niwa00004/meyerstj/data/SMAP/venv/lib/python3.11/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 52.86 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/nesi/nobackup/niwa00004/meyerstj/data/SMAP/venv/lib/python3.11/site-packages/distributed/client.py:3362: UserWarn

In [ ]:
zarrs = np.sort(list(Path('n5eil01u.ecs.nsidc.org/SMAP/SPL3SMP_E.006/zarr').glob('*.zarr')))

xr.open_mfdataset(zarrs, chunks = {}, engine='zarr')

# Here is a bit of EDA to justify why we have done things

In [ ]:
# open up one of the datasets.
ds = sa.extract_smap('2015-04-15', force = True, save = False)

# notice we have many "soil moisture variables."
ds

## Plot all the soil moisture variables

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import colormaps as cmaps
plt.style.use('ggplot')

subplot_kwargs = dict(transform =  ccrs.PlateCarree() , cmap = 'jet', cbar_kwargs = dict(shrink = 0.8, pad = 0.001, orientation = 'horizontal'))

fig, ax = plt.subplots(1,5, figsize = (20,50), subplot_kw = {'projection': ccrs.PlateCarree(central_longitude=171)}, layout='constrained')
data_var_list = ['soil_moisture', 'soil_moisture_scah','soil_moisture_scav','soil_moisture_dca_pm','soil_moisture_scah_pm' ]

for i, x in enumerate(ax):
    variable = data_var_list[i]
    ds[variable].plot(ax = x, **subplot_kwargs)
    x.set_title(variable)
    # x.set_extent([130,185, -10, -55])
    x.coastlines('10m')
plt.show()


SMAP documentation: <br><i>
"soil_moisture (soil_moisture_scah,soil_moisture_scav, soil_moisture_dca)
Daily global composite of the estimated soil
moisture at 9 km grid posting, as returned by
the L2_SM_P_E processing software. The
generic soil_moisture field is internally linked
to the output produced by the baseline
algorithm (DCA currently). "</i><br>
<br>
Okay... so let's extract the daily soil moisture from the am and pm channels. In the h5 file, I can only find `soil_moisture` (which was from the `AM` "group" of the `.h5` file), and `soil_moisture_dca_pm` (which was from the `PM` group of the `.h5` file). So, we take the <b>average of these two channels since they are the same (DCA) <b>, but just done at different times of day, i.e. the acending and descending passes. 

In [ ]:
ds_am  = ds[['soil_moisture']].assign_coords({'update':'am'}).expand_dims('update')
ds_pm  = ds[['soil_moisture_dca_pm']].rename(
    {'soil_moisture_dca_pm':'soil_moisture'}).assign_coords({'update':'pm'}).expand_dims('update')


In [ ]:
ds_daily = xr.concat([ds_am, ds_pm], dim = 'update').mean('update')



In [ ]:
fig, ax = plt.subplots(1,1, figsize = (20,50), subplot_kw = {'projection': ccrs.PlateCarree(central_longitude=171)}, layout='constrained')

ds_daily['soil_moisture'].plot(ax = ax, **subplot_kwargs)
ax.set_title('mean of dca_pm and soil_moisture am')
# x.set_extent([130,185, -10, -55])
ax.coastlines('10m')
plt.show()

In [ ]:
dates = pd.date_range('2015-04-01', '2015-04-05')
ease_grid = EASE2_grid(9000)

In [ ]:
%%time
delayed_list = []
for date in dates:
    delayed_list.append(
        delayed(extract_smap)(date, ease_grid, force = True, save = True)
    )

zarrs = compute(deåayed_list)

In [ ]:
import glob

paths = glob.glob('n5eil01u.ecs.nsidc.org/SMAP/SPL3SMP_E.006/*/*.h5')

zspec = [SingleHdf5ToZarr(str(p)) for p in paths]
# zarr_list = [ZarrToZarr(zarr_path).translate() for zarr_path in paths]

# # zspecs = [SingleHdf5ToZarr(str(p)).translate() for p in paths]
# json = MultiZarrToZarr(zarr_list, concat_dims = 'time', identical_dims = ['lat','lon']).translate()
# # concatenate_arrays(zarr_list)



In [ ]:
import h5py
import re
import glob

regex = re.compile("SMAP_L3_SM_P_E_(\d{8})_R\d{5}_\d{3}\.h5")

paths = glob.glob('n5eil01u.ecs.nsidc.org/SMAP/SPL3SMP_E.006/*/*.h5')

In [ ]:
def extract_h5_data(path : str):
    
    try:
        h5_file.close()
    except:
        pass
    h5_file = h5py.File(path, 'r')

    # json = SingleHdf5ToZarr(path).translate()
    json = SingleHdf5ToZarr(h5_file["Soil_Moisture_Retrieval_Data_AM"], url = path).translate()
    return json

In [ ]:
h5_file = h5py.File(paths[1], 'r')


h5_file["Soil_Moisture_Retrieval_Data_AM"].attrs

In [ ]:
zspec = MultiZarrToZarr(path = paths, indicts=json_list, coo_map={'time': regex}, identical_dims=['phony_dim_0', 'phony_dim_1']).translate()

In [ ]:
xr.open_dataset(zspec, engine = "kerchunk", group = "Soil_Moisture_Retrieval_Data_AM", chunks = {})

In [ ]:
xr.open_dataset(zspec_list[0], engine='kerchunk', chunks = {})

In [ ]:
zspec_list[-1]

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import colormaps as cmaps
plt.style.use('ggplot')

subplot_kwargs = dict(transform =  ccrs.PlateCarree() , cmap = cmaps.lipari_r, cbar_kwargs = dict(shrink = 0.8, pad = 0.001, orientation = 'horizontal'))

fig, ax = plt.subplots(1,2, figsize = (10,5), subplot_kw = {'projection': ccrs.PlateCarree()}, layout='constrained')

ds['soil_moisture'].plot(ax = ax[0], **subplot_kwargs)
ds_adjusted['soil_moisture'].plot(ax = ax[1],  **subplot_kwargs)

for x in ax:
    x.set_extent([160,180, -38, -60])
    x.coastlines('10m')
    
ax[0].set_title('Cropping EASE Grid')
ax[1].set_title('Creating linspace from EASE Grid')
plt.show()